In [57]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "Informer-Tensorflow/models")
sys.path.insert(1, "Informer-Tensorflow")
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

In [2]:
from attn import ProbAttention, AttentionLayer

In [3]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [4]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [5]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [6]:
def models():
    inputs = keras.layers.Input(shape=(2000,1), batch_size=8)
    x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(5)(x)
    return keras.Model(inputs, x)

In [7]:
model = models()

[TensorShape([8, 2000, 32]), TensorShape([8, 2000, 32]), TensorShape([8, 2000, 32])]


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(8, 2000, 1)]            0         
_________________________________________________________________
dense (Dense)                (8, 2000, 32)             64        
_________________________________________________________________
positional_embedding (Positi (8, 2000, 32)             0         
_________________________________________________________________
multi_head_prob_attention (M (8, 2000, 64)             10496     
_________________________________________________________________
conv_layer (ConvLayer)       (8, 666, 32)              6304      
_________________________________________________________________
layer_normalization (LayerNo (8, 666, 32)              64        
_________________________________________________________________
global_average_pooling1d (Gl (8, 32)                   0     

In [9]:
x = tf.ones([8,2000,1])
model(x)

<tf.Tensor: shape=(8, 5), dtype=float32, numpy=
array([[ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ],
       [ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ],
       [ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ],
       [ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ],
       [ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ],
       [ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ],
       [ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ],
       [ 0.023065  , -0.4200179 ,  1.1741719 ,  0.09310071,  1.7358772 ]],
      dtype=float32)>

In [10]:
data = np.load("data_shuffled.npz")
X = data["X"]
Y = data["Y"]



In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [15]:
learning_rate = CustomSchedule(d_model)
optimizer = keras.optimizers.experimential.AdamW(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

model.compile(optimizer="Adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [18]:
model.fit(x=X[0:904], y=Y[0:904], batch_size=8, epochs=500, validation_data=(X[904:], Y[904:]),
         callbacks=[tensorboard_callback])

Epoch 1/500
113/113 [==============================] - 10s 81ms/step - loss: 1.0560 - accuracy: 0.5166 - val_loss: 1.7469 - val_accuracy: 0.2292
Epoch 2/500
113/113 [==============================] - 7s 65ms/step - loss: 1.0357 - accuracy: 0.5221 - val_loss: 3.1543 - val_accuracy: 0.2083
Epoch 3/500
113/113 [==============================] - 8s 67ms/step - loss: 1.0378 - accuracy: 0.5122 - val_loss: 1.3557 - val_accuracy: 0.4479
Epoch 4/500
113/113 [==============================] - 7s 66ms/step - loss: 1.0328 - accuracy: 0.5100 - val_loss: 1.3043 - val_accuracy: 0.4479
Epoch 5/500
113/113 [==============================] - 7s 65ms/step - loss: 1.0273 - accuracy: 0.5277 - val_loss: 1.1229 - val_accuracy: 0.4583
Epoch 6/500
113/113 [==============================] - 7s 66ms/step - loss: 1.0564 - accuracy: 0.5133 - val_loss: 1.4587 - val_accuracy: 0.3438
Epoch 7/500
113/113 [==============================] - 8s 68ms/step - loss: 1.0622 - accuracy: 0.4834 - val_loss: 1.7632 - val_accuracy

113/113 [==============================] - 7s 65ms/step - loss: 0.8235 - accuracy: 0.6316 - val_loss: 9.9770 - val_accuracy: 0.2396
Epoch 113/500
113/113 [==============================] - 7s 65ms/step - loss: 0.8309 - accuracy: 0.6250 - val_loss: 4.0385 - val_accuracy: 0.4062
Epoch 114/500
113/113 [==============================] - 7s 64ms/step - loss: 0.8403 - accuracy: 0.6261 - val_loss: 1.9080 - val_accuracy: 0.3958
Epoch 115/500
113/113 [==============================] - 7s 66ms/step - loss: 0.8278 - accuracy: 0.6383 - val_loss: 6.3086 - val_accuracy: 0.2083
Epoch 116/500
113/113 [==============================] - 7s 65ms/step - loss: 0.8276 - accuracy: 0.6327 - val_loss: 1.8436 - val_accuracy: 0.4167
Epoch 117/500
113/113 [==============================] - 7s 65ms/step - loss: 0.8043 - accuracy: 0.6394 - val_loss: 4.5569 - val_accuracy: 0.1979
Epoch 118/500
113/113 [==============================] - 7s 65ms/step - loss: 0.7977 - accuracy: 0.6538 - val_loss: 10.4959 - val_accuracy

113/113 [==============================] - 7s 65ms/step - loss: 0.7050 - accuracy: 0.7400 - val_loss: 17.4891 - val_accuracy: 0.1979
Epoch 225/500
113/113 [==============================] - 7s 66ms/step - loss: 0.6697 - accuracy: 0.7423 - val_loss: 2.9948 - val_accuracy: 0.3542
Epoch 226/500
113/113 [==============================] - 7s 65ms/step - loss: 0.6622 - accuracy: 0.7544 - val_loss: 3.3361 - val_accuracy: 0.3750
Epoch 227/500
113/113 [==============================] - 7s 65ms/step - loss: 0.6900 - accuracy: 0.7389 - val_loss: 1.1578 - val_accuracy: 0.5208
Epoch 228/500
113/113 [==============================] - 7s 65ms/step - loss: 0.6726 - accuracy: 0.7456 - val_loss: 1.5403 - val_accuracy: 0.5417
Epoch 229/500
113/113 [==============================] - 7s 65ms/step - loss: 0.6699 - accuracy: 0.7423 - val_loss: 1.1755 - val_accuracy: 0.5000
Epoch 230/500
113/113 [==============================] - 7s 66ms/step - loss: 0.6850 - accuracy: 0.7588 - val_loss: 6.7004 - val_accuracy

113/113 [==============================] - 7s 64ms/step - loss: 0.6151 - accuracy: 0.7677 - val_loss: 1.3041 - val_accuracy: 0.4583
Epoch 337/500
113/113 [==============================] - 7s 64ms/step - loss: 0.5992 - accuracy: 0.7843 - val_loss: 3.4582 - val_accuracy: 0.4792
Epoch 338/500
113/113 [==============================] - 7s 65ms/step - loss: 0.5667 - accuracy: 0.7887 - val_loss: 13.1232 - val_accuracy: 0.2083
Epoch 339/500
113/113 [==============================] - 7s 64ms/step - loss: 0.5930 - accuracy: 0.7644 - val_loss: 1.8868 - val_accuracy: 0.4896
Epoch 340/500
113/113 [==============================] - 7s 65ms/step - loss: 0.5925 - accuracy: 0.7810 - val_loss: 2.2541 - val_accuracy: 0.3854
Epoch 341/500
113/113 [==============================] - 7s 64ms/step - loss: 0.6065 - accuracy: 0.7810 - val_loss: 2.6462 - val_accuracy: 0.4479
Epoch 342/500
113/113 [==============================] - 7s 64ms/step - loss: 0.6036 - accuracy: 0.7799 - val_loss: 7.3332 - val_accuracy

113/113 [==============================] - 7s 64ms/step - loss: 0.5437 - accuracy: 0.8064 - val_loss: 2.0935 - val_accuracy: 0.4062
Epoch 449/500
113/113 [==============================] - 7s 64ms/step - loss: 0.5601 - accuracy: 0.7942 - val_loss: 4.0669 - val_accuracy: 0.4167
Epoch 450/500
113/113 [==============================] - 7s 65ms/step - loss: 0.5550 - accuracy: 0.8009 - val_loss: 1.7885 - val_accuracy: 0.5208
Epoch 451/500
113/113 [==============================] - 7s 65ms/step - loss: 0.5545 - accuracy: 0.8031 - val_loss: 2.4838 - val_accuracy: 0.4479
Epoch 452/500
113/113 [==============================] - 7s 66ms/step - loss: 0.5545 - accuracy: 0.7976 - val_loss: 2.0465 - val_accuracy: 0.4271
Epoch 453/500
113/113 [==============================] - 7s 64ms/step - loss: 0.5245 - accuracy: 0.8031 - val_loss: 1.7093 - val_accuracy: 0.4792
Epoch 454/500
113/113 [==============================] - 7s 65ms/step - loss: 0.5381 - accuracy: 0.7998 - val_loss: 3.9881 - val_accuracy: